# DuCaPHEN HERA SARS-CoV-2 Bioinformatics workshop

Hi!

Welcome to the hands-on course part of the HERA bioinformatics workshop.


---
This course, and the view you're looking at right now, is a jupyter notebook. Which is a sequential process of executable code blocks. If you hover over a block then a "play-button" may appear on the left hand side of the screen. If you see this button then the block is executable. 

Once a block of code is finished with running, a small green checkmark will appear left next to the play-button.

---
### About the data that you will be analyzing
In this hands-on workshop you will *each* be processing 3 samples of a larger SARS-CoV-2 outbreak on Bonaire that happened near the end of 2022.


<!-- ---
### About the dataset(s)
The data that is analyzed throughout this course is publicly accessible data downloaded from the ENA (European Nucleotide Archive).  
The data that will be processed are raw FastQ reads both from an Illumina MiSeq sequencer as well as a Nanopore GridIon sequencer.  
The following Illumina dataset will be used:
https://www.ebi.ac.uk/ena/browser/view/ERR4082808?show=reads  
And the following Nanopore dataset will be used: https://www.ebi.ac.uk/ena/browser/view/ERR9900947?show=reads  

### About the analysis steps in this course

The analysis steps in this course are tailored for the analysis of SARS-CoV-2.  
These steps are based on the **SARS2seq** pipeline which is used for SARS-CoV-2 analysis within the RIVM. These various steps *can* be used to analyse other viruses as well, but success is not guaranteed.  

 -->


## Table of contents

1. Check if all your inputs are complete
2. Generate consensus genomes with the ViroConstrictor pipeline
3. Performing manual curation

# 1. Check if all your inputs are complete

Before you start with analyzing data, it is a good and often necessary practice to actually check if everything that you need is present.

You can press `Ctrl+B` on your keyboard to show or hide the directory and file structure that you will be working with. If you don't see any files on the left-hand side of your screen please press this button now.

When you can see the files please take a moment to ensure you have the following items ready:

##### Required files

* FastQ files  
  FastQ is the most common raw sequencing data format.  
  Within your file structure, you should have a folder called "input_files", within there should be 3 files named as `sample_x.fastq.gz`  
* A reference genome  
  To analyze the raw data and to create a consensus-genome it will be required to provide a reference genome to the software.  
  This file is named "sars_cov_2_reference.fasta" and should be present in the "resources" directory.
* Primer files  
  The data that will be analyzed is created with an amplification protocol, this means that primers (artificial pieces of DNA) were used during the sequencing protocol. For accurate processing it is now important that these pieces of artifical DNA will be removed. This information is provided through something that is called a BED file. There will be two primer files during this excercise.
  These files are named "articv3.bed" and "articv4.bed", both of these files should be present in the "resources" directory.

##### Optional (but helpful files)

* A genomic features file  
  To help the accuracy of the analysis tools, we can provide something called a genomic features file (GFF) to the analysis-toolset. This file corresponds to the earlier described reference genome and contains information such as open reading frames, gene positions and protein properties.  
  This file is named "sars_cov_2_features.gff" and should be present in the "resources" directory.

### Why is this Important?

Having all necessary files in the correct formats and in an organized manner ensures a smooth analysis process. It minimizes potential errors and delays, allowing you to focus on interpreting your results.

If you have any questions about file preparation, please don't hesitate to ask before we begin the analysis section of the course.


# 2. Generate consensus genomes with the ViroConstrictor pipeline

When you're sure that you have all the necessary files, then you can move on to actually analyzing the data. In this workshop we'll be using the ViroConstrictor pipeline to process the raw FastQ data and create consensus-genomes from this data.  
ViroConstrictor is a data-processing pipeline designed specifically for the analysis and consensus-genome generation of viral-sequencing data. 

Please find the documentation for ViroConstrictor [here](https://rivm-bioinformatics.github.io/ViroConstrictor/latest/manual/#run-a-single-target-analysis)

The base command is already typed out below, however the actual run-information still needs to be provided.
The information needs to be filled in for each section that says `<[your input here]>` and will be provided by the guiding presentation.

The analysis itself should take about 10 minutes to complete.

In [ ]:
%%bash
source activate pipeline_env

viroconstrictor -i input_files/ -o pipeline_output \
    --platform <[your input here]> \
    --amplicon-type end-to-end \
    --reference <[your input here]> \
    --primers <[your input here]> \
    --features <[your input here]> \
    --min-coverage <[your input here]> \
    --target <[your input here]>

In [ ]:
##backup command, only to be used by instructor
%%bash
source activate pipeline_env

viroconstrictor -i input_files/ -o pipeline_output      \
    --platform nanopore                                 \
    --amplicon-type end-to-end                          \
    --reference ./resources/sars_cov_2_reference.fasta  \
    --primers resources/articv4.bed                     \
    --features resources/sars_cov_2_features.gff        \
    --min-coverage 50                                   \
    --target sars-cov-2

# 3. Check the quality control reports

Once the ViroConstrictor pipeline is done with processing the data, we should check if everything in the analysis went accordingly.

The pipeline already does the most of the heavy-lifting when it comes to quality-control. However, it is still necessary to manually check if there are no weird outliers.

While processing the data, the ViroConstrictor pipeline creates a [MultiQC](https://multiqc.info/) report. MultiQC is a powerful tool that aggregates (quality-related) results from all kinds of bioinformatics tools into a single report for easy manual checking.

While a Quality Control report may not be _always_ the most useful, it can still provide crucial information. Especially when a laboratory, or its personell, is relatively new to sequencing.

In [ ]:
# <-- Click here to open a browser window and view the quality control report
!`cmd.exe /C start msedge.exe file://wsl.localhost/Ubuntu/$(realpath pipeline_output/results/multiqc.html) >> /dev/null 2>&1`

# 4. Check the alignment files with IGV

When it is certain that there are not egregious quality issues with both the raw and processed data, it is often good practice to check the BAM files for any obvious errors.

A BAM file, also called a Binary Alignment Map, is a representation of the sequencing data when it is aligned to a reference genome. You can compare this to a multiple sequence alignment, but in a massively larger scale and with way more statistics and math involved.

It's not possible to **view** BAM files without special tools as this file is created in such a way that makes it very efficient for computers but not for humans. Luckily special tools exist, such as [IGV](https://igv.org/) which we will be using here.

Looking at the alignments is a very good method to view very obvious, or very well hidden errors, and IGV is maybe one of the strongest tools to visualise the intermediate files of the pipeline.

In [ ]:
# <-- Click here to open the IGV alignment viewer to check the alignments
!source activate pipeline_env; igv -g resources/sars_cov_2_reference.fasta pipeline_output/data/Virus\~sars-cov-2/RefID\~MN908947.3/alignment/bam-files/sample_*.bam pipeline_output/data/Virus\~sars-cov-2/RefID\~MN908947.3/primers/sample_*_removedprimers.bed -l MN908947.3:12,421-14,775 >> /dev/null 2>&1

# 5. Check the consensus validity with NextClade

Now that the consensus sequence is generated from the Nanopore data we have to perform a manual curation step to see if everything that we have done resulted in a proper consensus genome.  

The manual curation of the generated consensus sequences can be done the most easily with the web-based tool [NextClade](https://clades.nextstrain.org/).

This can be done by dragging the consensus genomes, which are available in the `pipeline_output/results/Virus~sars-cov-2/RefID~MN908947.3/` directory.

By executing the codeblock below you will open the windows explorer on the location where the consensus genomes are available and drag it to NextClade.

In [2]:
# <-- Click here to open the file explorer to view the output files
import warnings
import subprocess
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    subprocess.run("explorer.exe $(wslpath -w ./pipeline_output/results/Virus~sars-cov-2/RefID~MN908947.3/)", shell=True, stdout=None, stderr=None)